<a href="https://colab.research.google.com/github/hariprasad7/hari_INFO5731_Spring2021/blob/main/In_class_exercise_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The seventh in-class-exercise (20 points in total, 3/16/2021)**

Question description: In the last in-class-exercise (exercise-06), you collected the titles of 100 articles about data science, natural language processing, and machine learning. The 100 article titles will be used as the text corpus of this exercise. Perform the following tasks:

(1) (8 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
#Gensim,plotting tools,logging
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
!pip install pyLDAvis
!pip install --upgrade gensim
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

     |████████████████████████████████| 1.7MB 5.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=893e7b4110d04f2cc9854b3a1ffe83a40613e0f7ece29fdd37e3c3de6ddf5717
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis
     |████████████████████████████████| 24.2MB 1.4MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')               #stopwords
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import pandas as pd
df = pd.read_csv("/content/Inclass6.csv")
df.head(100)

,Unnamed: 0,0
0,0,\n Privacy Preserving Data Mi...
1,1,\n \t Privacy-Preserving Dat...
2,2,\nData Mining: Concepts and Techniques\n ...
3,3,\n The WEKA Data Mining Softw...
4,4,\n Efficient and Effective Cl...
...,...,...
95,95,\n Normalization of Data in D...
96,96,\n Knowledge Discovery in Dat...
97,97,\nData Mining in Social Networks\n ...
98,98,\n Materialized Views in Data...


In [15]:
import re
df.columns=["Index", "Title"]
df['Title']=df['Title'].str.lower()
d = df['Title'].values.tolist() 
d= [re.sub(" $","",se) for se in d]
d = [re.sub('\S*@\S*\s?', '', se) for se in d]
d= [re.sub('\s+', ' ', se) for se in d]
d = [re.sub("\'", "", se) for se in d]
print(d[:100])

[' privacy preserving data mining ', ' privacy-preserving data mining ', ' data mining: concepts and techniques ', ' the weka data mining software: an update ', ' efficient and effective clustering methods for spatial data mining ', ' data mining: an overview from database perspective ', ' from data mining to knowledge discovery in databases. ', ' data mining approaches for intrusion detection, ', ' survey of clustering data mining techniques ', ' the elements of statistical learning -- data mining, inference, and prediction ', ' untangling text data mining ', ' fastmap: a fast algorithm for indexing, data-mining and visualization of traditional and multimedia datasets ', ' sprint: a scalable parallel classifier for data mining ', ' astrolabe: a robust and scalable technology for distributed system monitoring, management, and data mining ', ' limiting privacy breaches in privacy preserving data mining ', ' data mining ', ' graph-based data mining ', ' data mining ', ' data mining ', ' 

In [ ]:
from gensim import corpora
from gensim import models
from gensim.models import LdaModel
from gensim.models import TfidfModel
from gensim.models import CoherenceModel
from gensim import models
def setowor(sen):
    for sentence in sen:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
d_words = list(setowor(d))
print(d_words[:100])

In [ ]:
#Bigram and Trigram models
bi_gram = gensim.models.Phrases(d_words, min_count=5, threshold=100) # higher threshold fewer phrases.
tri_gram = gensim.models.Phrases(bi_gram[d_words], threshold=100)  
bi_mod = gensim.models.phrases.Phraser(bi_gram)
tri_mod = gensim.models.phrases.Phraser(tri_gram)
print(tri_mod[bi_mod[d_words[1]]])

In [ ]:
# Defining functions for stopwords, Bigrams, Trigrams and Lemmatization
def rem_stopwords(t):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in t]
def makbigrams(t):
    return [bi_mod[doc] for doc in t]
def maktrigrams(t):
    return [tri_mod[bi_mod[doc]] for doc in t]
def lemmat(t, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts = []
    for sent in t:
        doc = nlp(" ".join(sent)) 
        texts.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts
d_wordsnostops = rem_stopwords(d_words)        # Removal of Stop Words and form Bigrams
d_wordsbi_grams = makbigrams(d_wordsnostops)
nlp = spacy.load('en', disable=['parser', 'ner'])
d_lemma = lemmat(d_wordsbi_grams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])        #Lemmatization
print(d_lemma[:1])

In [ ]:
id2word = corpora.Dictionary(d_lemma)           #Dictionary, corpus, tdf and view
t = d_lemma
corpus = [id2word.doc2bow(text) for text in t]
print(corpus[:1])

In [ ]:
res = [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:]]
LDAmodel = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=20,random_state=100,update_every=1,chunksize=100,passes=20,alpha='auto',per_word_topics=True)
pprint(LDAmodel.print_topics())                  # Keyword in 20 topics
docLDA = LDAmodel[corpus]

In [ ]:
print('\nPerplexity: ', LDAmodel.log_perplexity(corpus))   #perplexity
coherence_model_lda = CoherenceModel(model=LDAmodel, texts=d_lemma, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)  

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
import os     
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       
install_java()
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [ ]:
def compcoherenceval(dictionary, corpus, texts, start, limit, step):
    
    coherenceval= []
    modelli= []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(malletpath, corpus=corpus, num_topics=num_topics, id2word=id2word)
        modelli.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherenceval.append(coherencemodel.get_coherence())

    return modelli, coherenceval
modelli, coherenceval = compcoherenceval(dictionary=id2word, corpus=corpus, texts=d_lemma, start=2, limit=40, step=6)

In [ ]:
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(y, coherenceval)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherenceval"), loc='best')
plt.show()                                     #Graph

In [ ]:
for l, v in zip(y, coherenceval):
    print("Num of Topics =", l, " has Coherence Value:", round(v, 4))

In [ ]:
optimal_model = modelli[5]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))     

In [ ]:
def ftopicssentences(ldamodel=LDAmodel, corpus=corpus, texts=d):
    sent_topics_df = pd.DataFrame()

    
    for i, row in enumerate(ldamodel[corpus]):           #For main topic in each document
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant Topic', 'Perc_Contribution', 'Topic Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = ftopicssentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document Num', 'Dominant Topic', 'Topic Perc_Contribution', 'Keywords', 'Texts']

# Show
df_dominant_topic.head(100)

In [ ]:
topicssorteddf_mallet = pd.DataFrame()                     #Top 5

topicsoutdf_grpd = df_topic_sents_keywords.groupby('Dominant Topic')

for i, grp in topicsoutdf_grpd:
    topicssorteddf_mallet = pd.concat([topicssorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

   
topicssorteddf_mallet.reset_index(drop=True, inplace=True)

topicssorteddf_mallet.columns = ['Topic No', "Topic Perc_Contrib", "Keywords", "Texts"]    #Format

topicssorteddf_mallet.head()

## (2) (8 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [22]:
# Write your code here
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
def load_data(path,file_name):
    doc_list = []
    titles=[]
    with open( os.path.join(path, file_name) ,"r") as f:
        for line in f.readlines():
            text = line.strip()
            doc_list.append(text)
    print("Total No: of Documents:",len(doc_list))
    titles.append( text[0:min(len(text),100)] )
    return doc_list,titles


In [23]:
def preprocess_data(doc_set):
  
    tokenizer = RegexpTokenizer(r'\w+')
    en_stop = set(stopwords.words('english'))
    p_stemmer = PorterStemmer()
    texts = []
    for i in doc_set:
        raw = i.lower()   #cleaning
        tokens = tokenizer.tokenize(raw)
        stopped_tokens = [i for i in tokens if not i in en_stop]
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        texts.append(stemmed_tokens)   #Preprocessed text
    return texts  

In [24]:
def prepare_corpus(clean_doc):
    dictionary = corpora.Dictionary(clean_doc)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_doc]
    return dictionary,doc_term_matrix

In [25]:
def create_gensim_lsa_model(clean_doc,number_of_topics,words):
    dictionary,doc_term_matrix=prepare_corpus(clean_doc)
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel        #LSA model

In [26]:
def compcoherencevalues(dictionary, doc_term_matrix, clean_doc, stop, start=2, step=3):
    coherencevalues = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=clean_doc, dictionary=dictionary, coherence='c_v')
        coherencevalues.append(coherencemodel.get_coherence())
    return model_list, coherencevalues

In [ ]:
def plot_graph(clean_doc,start, stop, step):
    dictionary,doc_term_matrix=prepare_corpus(clean_doc)
    model_list, coherencevalues = compcoherencevalues(dictionary, doc_term_matrix,clean_doc,
                                                            stop, start, step)
    x = range(start, stop, step)
    plt.plot(x, coherencevalues)
    plt.xlabel("Number of Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence values"), loc='best')
    plt.show()                                   #Graph
start,stop,step=2,14,1
plot_graph(clean_text,start,stop,step)

## (3) (4 points) Compare the results generated by the two topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)
'''
As we all know, LDA is used to assign a subject to text in a document. It creates a topic per document and word per topic model, both of which are modeled as Dirichlet distributions are a type of distribution. LSA is a computer program that analyzes text and creates a report. From a corpus of text, create a semantic space. In this case, LDA focuses on resolving a problem. LSA focuses on decreasing matrix dimension, while modeling focuses on solving problems. When it comes to contrasting
'''